In [125]:
using LinearAlgebra
using PlotlyJS
using FFTW
using BenchmarkTools
using Kronecker
using Primes

In [126]:
vecsize = 9
x = randn(vecsize) + 1im * randn(vecsize);

In [127]:
# Functions to build a DFT matrix for prime sizes

function dft_matrix(n)
    """ 
    Computes the Discrete Fourier Transform matrix of size n.
    Based on algorithm 1.16 from
    Van Loan, C. (1992). Computational frameworks for the fast Fourier transform.
    
    Input: 
    n: (integer)
    
    Returns:
    F: (n x n complex matrix)
    """
    
    # Base cases
    if n == 1
        return 1
    end
    
    F = ones(ComplexF64, n, n)
    F[1, 2] = 1
    
    for p = 1:n-1
        F[p + 1, 2] = exp(-2 * pi * 1im * p/n)
    end
    
    for q = 2:n-1
        F[:, q + 1] = F[:, q] .* F[:, 2]
    end

    return F
end

function naive_dft(x)
    """ 
    Computes the Discrete Fourier Transform of an input x.
    
    Input: 
    x: (vector)
    
    Returns:
    y: (vector)
    """
    
    return dft_matrix(length(x)) * x
end

function naive_idft_unscaled(y)
    """ 
    Computes the inverse Discrete Fourier Transform of an input y (unscaled).
    
    Input: 
    y: (vector)
    
    Returns:
    x: (vector)
    """
    
    return (dft_matrix(length(y))' * y) 
end

naive_idft_unscaled (generic function with 1 method)

In [128]:
function bit_reverse(i, bits)
    r = 0
    for _ in 1:bits
        r = (r << 1) | (i & 1)  # Shift left and add least significant bit
        i >>= 1                 # Shift input right
    end
    return r
end


function cooley_tukey_perm(n, return_matrix = false)
    """ 
    Finds the Cooley Tukey algorithm permutation matrix using bitwise operations.
    
    Input: 
    n: (integer, size of fft)
    
    Returns:
    P: (permutation matrix)
    """ 
    

    t = Int(log2(n))
    perm = bit_reverse.(collect(0:n-1), t) .+1
    
    if !return_matrix
        return perm
    end
    
    P = I + zeros(n, n)
    
    return P[:, perm] 
end

function pre_compute_roots_unity(n, inverse = false)
    """ 
    Precomputes the roots of unity array (Ω) for Pease FFT/IFFT based on (1.8.4)
    from Van Loan, C. (1992).
    """
    t = Int(log2(n))
    pwr_sign = inverse ? 1 : -1
    Ω = ones(ComplexF64, n ÷ 2, t)  
        
    for q in 1:t
        L = 2^q
        L_star = L ÷ 2
        r = n ÷ L

        for j in 0:L_star-1, k in 0:r-1
            Ω[j * r + k + 1, q] = cos(2 * pi * j / L) + pwr_sign * 1im * sin(2 * pi * j/L)
        end
    end
    
    return Ω
end


pre_compute_roots_unity (generic function with 2 methods)

In [129]:
function pease_fft_base(z, inverse = false)
    """ 
    Computes the Pease FFT/IFFT based on Algorithm 1.8.1 
    from Van Loan, C. (1992).
    """

    n = length(z)
    t = Int(log2(n))
    m = n ÷ 2
    p = cooley_tukey_perm(n)

    # Applying permutation
    x = copy(z)[p]  
    y = copy(x)  
    Ω = pre_compute_roots_unity(n, inverse)  

    # Performing Pease FFT
    for q = 1:t
        x, y = y, x  
        u = Ω[:, q] .* y[2:2:n]  

        x[1:m] = y[1:2:n] + u
        x[m + 1:n] = y[1:2:n] - u
    end

    # Scale result for inverse FFT
    return inverse ? x ./ n : x
end 

function pease_fft(x)
    return pease_fft_base(x, false)
end

function pease_ifft(y)
    return pease_fft_base(y, true)
end


pease_ifft (generic function with 1 method)

In [9]:
norm(pease_ifft(pease_fft(x)) - x, Inf)/norm(x, Inf)

2.5060156982423103e-16

In [ ]:
# Function to perform Mixed Radix recursive FFT  
function genfft_base(x, n, inverse = false)
    """ 
    Computes the Radix recursive FFT/IFFT for a vector with general size
    based on Algorithm present 
    in Section 2.1.4 
    from Van Loan, C. (1992).
    """

    pwr_sign = inverse ? 1 : -1
    
    omega = exp(pwr_sign * 2im * π / n)

    # Base case: Use DFT matrix multiplication if n is prime
    if isprime(n)
        return inverse ? naive_idft_unscaled(x) : naive_dft(x)
    end

    # Find smallest factor p of n
    p = first(filter(d -> n % d == 0, 2:n))
    m = n ÷ p  # Compute m such that n = p * m

    # Twiddle factor vector 
    Omega_vec = [omega^j for j in 0:m-1]  

    # Step 1: Compute FFTs along p smaller groups
    z = similar(x, ComplexF64)  
    for j in 0:p-1
        z[j*m+1:(j+1)*m] .= (Omega_vec .^ j) .* genfft_base(x[j+1:p:end], m, inverse)  # Element-wise multiplication
    end

    # Step 2: Compute FFTs along m groups
    y = similar(x, ComplexF64)  
    for j in 0:m-1
        y[j+1:m:end] .= genfft_base(z[j+1:m:end], p, inverse)  # Vectorized slicing
    end

    return y
end

function genfft(x)
    n = length(x)
    return genfft_base(x, n)
end

function genifft(x)
    n = length(x)
    return genfft_base(x, n, true) / n
end

genifft (generic function with 1 method)

In [131]:
norm(fft(x) - genfft(x), Inf)/norm(fft(x), Inf)

6.03076649089214e-16

In [132]:
norm(x - genifft(genfft(x)), Inf)/norm(x, Inf)

4.751372675741167e-16